In [1]:
%matplotlib inline
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import matplotlib
import time
import datetime
import pandas
import numpy
import scipy.stats
import abcutils
import seaborn

In [4]:
TEST_PLATFORMS = [
    'scratch1@edison',
    'scratch2@edison',
    'scratch3@edison',
    'cscratch@cori-knl',
    'cscratch@cori-haswell',
    'mira-fs1@mira'
]

## Load and Synthesize Data from CSV

In [5]:
df = pandas.concat([abcutils.load_and_synthesize_csv('summaries/edison-summaries_2017-02-14-2017-12-30.csv', system='edison'),
                    abcutils.load_and_synthesize_csv('summaries/cori-summaries_2017-02-14-2017-12-31.csv', system='cori'),
                    abcutils.load_and_synthesize_csv('summaries/alcf-tokio-results-2_14_17-2_15_18.csv', system='mira')],
                   axis='rows')

In [ ]:
platforms = [ 'scratch1@edison', 'cscratch@cori-knl', 'mira-fs1@mira' ]
benchmarks = [ 'ior_shared_write', 'ior_fpp_write', 'hacc_io_write_fpp_write', 'vpicio_uni_shared_write',
               'ior_shared_read', 'ior_fpp_read', 'hacc_io_read_fpp_read', 'dbscan_read_shared_read' ]
plot_metric = 'darshan_normalized_perf_by_max'
date_start = datetime.datetime(2017, 2, 14)
date_end = date_start + datetime.timedelta(days=365)

filtered_df = df[(df['_datetime_start'] < date_end) &
                 (df['_datetime_start'] >= date_start)]

job_times = {}
heatmap_dat = {}
platform_groups = filtered_df.groupby('_test_platform')
for platform_id in platforms:
    platform_df = platform_groups.get_group((platform_id))
    job_ids = sorted(list(set(platform_df['_jobid'].values)))
    job_times[platform_id] = [platform_df[(platform_df['_jobid'] == jobid)]['_datetime_start'].values[0] for jobid in job_ids]
    heatmap_dat[platform_id] = pandas.DataFrame(0.0, index=job_ids, columns=benchmarks)
    benchmark_groups = platform_df.groupby(by='_benchmark_id')
    for benchmark_id in benchmarks:
        benchmark_df = benchmark_groups.get_group((benchmark_id)).set_index('_jobid')
        heatmap_dat[platform_id][benchmark_id] = benchmark_df[plot_metric]

In [ ]:
fig, axes = matplotlib.pyplot.subplots(nrows=len(heatmap_dat), ncols=1)
fig.set_size_inches(20,5*len(heatmap_dat))
#matplotlib.pyplot.locator_params(nticks=4)

i = 0
for platform_id in heatmap_dat:
    tmp_df = heatmap_dat[platform_id].transpose()
    #tmp_df['_datetime_start'] = job_times[platform_id]
    #tmp_df = tmp_df.set_index('_datetime_start')
    #tmp_df = tmp_df.transpose()

    seaborn.heatmap(tmp_df, ax=axes[i])
    axes[i].set_title(platform_id)
    #axes[i].xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(10))
    axes[i].xaxis.label.set_visible(False)
    axes[i].set_xticks([])
    #axes[i].xaxis.set_major_locator(matplotlib.dates.DayLocator(interval=1))
    #axes[i].xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%Y-%m-%d'))
    axes[i].get_figure().subplots_adjust(hspace=.2)
    i += 1

#fig.autofmt_xdate()

pass